In [117]:
import xmltodict
from bs4 import BeautifulSoup
import pprint
import os
import re
import pandas as pd

In [118]:
image_folder = os.path.join(os.getcwd(),'IMAGE_DATA')

In [119]:
image_root = os.path.join(image_folder,os.listdir(image_folder)[0])

In [120]:
image_root_lst = os.listdir(image_root)

In [121]:

r = re.compile('(Index)')
# re.findall(r,image_root_lst)
idx_file = list(filter(r.findall,image_root_lst))[0]


In [122]:
with open(os.path.join(image_root,idx_file),'r') as f:
    index_xml = f.read()

In [123]:
idx_xml = BeautifulSoup(index_xml,'xml')

In [124]:
# ind_file.find('<Col>'==22)
# ind_file.find_all('Wells', {'Col': [22]})

In [125]:
idx_dict = xmltodict.parse(index_xml)

In [126]:
# pprint.pprint(idx_dict['EvaluationInputData'])

In [127]:
idx_dict['EvaluationInputData'].keys()

dict_keys(['@Version', '@xmlns', '@xmlns:xsd', '@xmlns:xsi', 'User', 'InstrumentType', 'Plates', 'Wells', 'Maps', 'Images'])

In [132]:
test_df = pd.DataFrame.from_dict(idx_dict['EvaluationInputData']['Wells']['Well'][0],orient='columns')

In [133]:
test_df

,id,Col,Row,SublayoutID,Image
0,0101,1,1,1,{}
1,0101,1,1,1,{'@id': '0101K1F1P1R2'}
2,0101,1,1,1,{'@id': '0101K1F2P1R1'}
3,0101,1,1,1,{'@id': '0101K1F2P1R2'}
4,0101,1,1,1,{'@id': '0101K1F3P1R1'}
5,0101,1,1,1,{'@id': '0101K1F3P1R2'}


In [130]:
image_id_lst = []
for i in range(len(idx_dict['EvaluationInputData']['Wells']['Well'][0]['Image'])):
    image_id_lst.append(idx_dict['EvaluationInputData']['Wells']['Well'][i]['Image'][i].pop('@id'))

In [131]:
image_id_lst

['0101K1F1P1R1',
 '0201K1F1P1R2',
 '0301K1F2P1R1',
 '0401K1F2P1R2',
 '0501K1F3P1R1',
 '0601K1F3P1R2']

In [134]:
test_df['Image'] = image_id_lst

In [135]:
test_df

,id,Col,Row,SublayoutID,Image
0,0101,1,1,1,0101K1F1P1R1
1,0101,1,1,1,0201K1F1P1R2
2,0101,1,1,1,0301K1F2P1R1
3,0101,1,1,1,0401K1F2P1R2
4,0101,1,1,1,0501K1F3P1R1
5,0101,1,1,1,0601K1F3P1R2


In [ ]:


# # r = re.compile('.*r04c11')
# r = re.compile('(Index)')

# with open (list(filter(r.findall,im_lst))[0],'r') as f:
#     z = f.read()
# z


# import numpy as np
# samp = np.array([21,22,33,4,5,6,7,87])
# mask = np.array([0,0,0,1,1,1,1,0],dtype=bool)
# samp[mask]

